## Packages

In [1]:
# General
import pandas as pd
from pprint import pprint

# Google News API
# This library will be used to parse the JSON data returned by the API.
import json
# This library will be used to fetch the API.
import urllib.request

# Google News API - FinBERT Sentiment model
import nltk
import torch
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSequenceClassification, AutoTokenizer

nltk.download("punkt")

# GlassDoor - RoBERTa Sentiment model
from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[nltk_data] Downloading package punkt to /Users/hpatel31/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data

In [5]:
from IPython.display import display, HTML

### Google News API

In [21]:
# https://docs.python.org/3/library/json.html
# This library will be used to parse the JSON data returned by the API.
import json
# https://docs.python.org/3/library/urllib.request.html#module-urllib.request
# This library will be used to fetch the API.
import urllib.request
#company = "Burger%20King"
#company = "Octopus%20energy"
company = "Octopus%20Investments"
#company = "Causaly"
apikey = "d6ceaba74589c370d566dea0b051fc61"
#url = f"https://gnews.io/api/v4/search?q=google&lang=en&country=us&max=20&apikey={apikey}"
url = f"https://gnews.io/api/v4/search?q={company}&lang=en&country=au&max=10&apikey={apikey}"
#url = f"https://gnews.io/api/v4/search?q=Octopus%20Investments&lang=en&country=gb&max=100&apikey={apikey}"

with urllib.request.urlopen(url) as response:
    data = json.loads(response.read().decode("utf-8"))
    articles = data["articles"]

    title = []
    description = []

    for i in range(len(articles)):
        # articles[i].title
        title.append(articles[i]['title'])
        # articles[i].description
        description.append(articles[i]['description'])
        # You can replace {property} below with any of the article properties returned by the API.
        # articles[i].{property}
        #print(f"{articles[i]['{property}']}")

        # Delete this line to display all the articles returned by the request. Currently only the first article is displayed.
        #break
        
#Octopus Investments

In [22]:
google_octopus_investments = pd.DataFrame(list(zip(title, description)),
               columns =['Title', 'Description'])
google_octopus_investments["Text"] = [f"{a}. {b}" for a,b in zip(google_octopus_investments.Title,google_octopus_investments.Description)]

display(HTML(google_octopus_investments.to_html()))

,Title,Description,Text
0,Octopus Australia buys Dulacca Wind Farm,"Octopus Investments has made its fourth investment in Australia, acquiring the Dulacca Wind Farm from global renewable energy company RES.","Octopus Australia buys Dulacca Wind Farm. Octopus Investments has made its fourth investment in Australia, acquiring the Dulacca Wind Farm from global renewable energy company RES."
1,Brisbane couple builds tech giant on the quiet,The hunt for the next Atlassian could be over after Paul and Sonia Stovell’s little-known Octopus Deploy banked one of the biggest ever local tech VC investments.,Brisbane couple builds tech giant on the quiet. The hunt for the next Atlassian could be over after Paul and Sonia Stovell’s little-known Octopus Deploy banked one of the biggest ever local tech VC investments.


### GlassDoor Reviews

In [14]:
octopus_investments_reviews = pd.read_csv("/Users/hpatel31/Library/CloudStorage/OneDrive-KPMG/Documents/octopus_investments_reviews.csv")
octopus_investments_reviews["headline"] = octopus_investments_reviews["headline"].astype(str)
octopus_investments_reviews["reviewText"] = [f"{a}. {b}. {c}" for a,b,c in zip(octopus_investments_reviews.headline,octopus_investments_reviews.pros,octopus_investments_reviews.cons)]
octopus_investments_reviews.drop(octopus_investments_reviews.columns[[5]], axis=1, inplace = True)
display(HTML(octopus_investments_reviews.to_html()))

,firm,rating,headline,pros,cons,reviewText
0,Octopus Investments,5,fantastic work place,"A great company, which gives you an opportunity to learn and develop quickly. Extremely friendly colleagues and lovely work environment.",Pay will always be lower compared to other similar sized companies.,"fantastic work place. A great company, which gives you an opportunity to learn and develop quickly. Extremely friendly colleagues and lovely work environment.. Pay will always be lower compared to other similar sized companies."
1,Octopus Investments,5,"Great people, poor pay","Good place to work. Not a lot of pressure, good people, good holiday entitlement",Pay is pretty low for the industry.,"Great people, poor pay. Good place to work. Not a lot of pressure, good people, good holiday entitlement. Pay is pretty low for the industry."
2,Octopus Investments,5,Great culture,You couldn't work with a better group of people than at Octopus. The culture is A+. Benefits are great.,Pay is below market rate. Management abilities amongst senior leaders are mixed.,Great culture. You couldn't work with a better group of people than at Octopus. The culture is A+. Benefits are great.. Pay is below market rate. Management abilities amongst senior leaders are mixed.
3,Octopus Investments,4,"Good company except low pay, poor middle management",Can be good progression Lots of things happening,Will happily admit they underpay Middle management is hit and miss,"Good company except low pay, poor middle management. Can be good progression Lots of things happening. Will happily admit they underpay Middle management is hit and miss"
4,Octopus Investments,5,Great company to start career at,"Fast paced, learn a lot. There is the ability to get into a fun progressive team that wants to change its industry.",Had an average manager in the end. But was good before then.,"Great company to start career at. Fast paced, learn a lot. There is the ability to get into a fun progressive team that wants to change its industry.. Had an average manager in the end. But was good before then."
5,Octopus Investments,1,"Low pay, no loyalty","Friendly staff, social activities, work life balance","Low pay, no loyalty to staff which casts doubt on their true B Corp status","Low pay, no loyalty. Friendly staff, social activities, work life balance. Low pay, no loyalty to staff which casts doubt on their true B Corp status"
6,Octopus Investments,1,Good start,"Life balance, opportunities, people, colleagues","Management, development, not enough progression","Good start. Life balance, opportunities, people, colleagues. Management, development, not enough progression"
7,Octopus Investments,3,Average. Good stepping stone to better firms,After work drinks. Nice building. Desks that go up and down (exciting stuff).,Senior and mid-level management.\nCorporate identity.\nZero cohesiveness between the different departments / companies.\nC-Level board haven't got a clue on what's really going on or they just don't care.,Average. Good stepping stone to better firms. After work drinks. Nice building. Desks that go up and down (exciting stuff).. Senior and mid-level management.\nCorporate identity.\nZero cohesiveness between the different departments / companies.\nC-Level board haven't got a clue on what's really going on or they just don't care.
8,Octopus Investments,5,Amazing place to work!,"Octopus has such a great energy about it as a business, its people are dynamic, driven but also friendly and transparent. The energy within the business means that you are constantly learning, trying new things and solving problems - and are given lots of autonomy and creativity to do so. Outside of this, the company offers a clear and solid path for development, with a structured process for annual reviews and promotions - it is somewhere that enables you to have a long and varied career.","I personally love the non-corporate, non-hierachial ways of working, but for someone coming from corporate finance it ma

## Octopus Investment GlassDoor Review - Sentiment Analysis

In [11]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta)
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', 'Positive']

def roberta_sentiment(
    text: str
):
    encoded_text = roberta_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    output = roberta_model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    return scores

In [15]:
# Sentiment Analysis for each review of Octopus Investments
neg = []
neu = []
pos = []
for text in octopus_investments_reviews["headline"]:
    output = roberta_sentiment(text = text)
    neg.append(output[0])
    neu.append(output[1])
    pos.append(output[2])

octopus_investments_reviews["Negative Sentiment"] = neg
octopus_investments_reviews["Neutral Sentiment"] = neu
octopus_investments_reviews["Positive Sentiment"] = pos

sentiment_df = octopus_investments_reviews[["Negative Sentiment", "Neutral Sentiment", "Positive Sentiment"]]

octopus_investments_reviews['Overall Sentiment'] = sentiment_df.idxmax(axis=1)
display(HTML(octopus_investments_reviews.to_html()))

,firm,rating,headline,pros,cons,reviewText,Negative Sentiment,Neutral Sentiment,Positive Sentiment,Overall Sentiment
0,Octopus Investments,5,fantastic work place,"A great company, which gives you an opportunity to learn and develop quickly. Extremely friendly colleagues and lovely work environment.",Pay will always be lower compared to other similar sized companies.,"fantastic work place. A great company, which gives you an opportunity to learn and develop quickly. Extremely friendly colleagues and lovely work environment.. Pay will always be lower compared to other similar sized companies.",0.014907,0.107335,0.877758,Positive Sentiment
1,Octopus Investments,5,"Great people, poor pay","Good place to work. Not a lot of pressure, good people, good holiday entitlement",Pay is pretty low for the industry.,"Great people, poor pay. Good place to work. Not a lot of pressure, good people, good holiday entitlement. Pay is pretty low for the industry.",0.102838,0.251001,0.646161,Positive Sentiment
2,Octopus Investments,5,Great culture,You couldn't work with a better group of people than at Octopus. The culture is A+. Benefits are great.,Pay is below market rate. Management abilities amongst senior leaders are mixed.,Great culture. You couldn't work with a better group of people than at Octopus. The culture is A+. Benefits are great.. Pay is below market rate. Management abilities amongst senior leaders are mixed.,0.006009,0.086428,0.907563,Positive Sentiment
3,Octopus Investments,4,"Good company except low pay, poor middle management",Can be good progression Lots of things happening,Will happily admit they underpay Middle management is hit and miss,"Good company except low pay, poor middle management. Can be good progression Lots of things happening. Will happily admit they underpay Middle management is hit and miss",0.298008,0.430289,0.271703,Neutral Sentiment
4,Octopus Investments,5,Great company to start career at,"Fast paced, learn a lot. There is the ability to get into a fun progressive team that wants to change its industry.",Had an average manager in the end. But was good before then.,"Great company to start career at. Fast paced, learn a lot. There is the ability to get into a fun progressive team that wants to change its industry.. Had an average manager in the end. But was good before then.",0.002196,0.037784,0.960020,Positive Sentiment
5,Octopus Investments,1,"Low pay, no loyalty","Friendly staff, social activities, work life balance","Low pay, no loyalty to staff which casts doubt on their true B Corp status","Low pay, no loyalty. Friendly staff, social activities, work life balance. Low pay, no loyalty to staff which casts doubt on their true B Corp status",0.655419,0.314521,0.030060,Negative Sentiment
6,Octopus Investments,1,Good start,"Life balance, opportunities, people, colleagues","Management, development, not enough progression","Good start. Life balance, opportunities, people, colleagues. Management, development, not enough progression",0.006099,0.087203,0.906698,Positive Sentiment
7,Octopus Investments,3,Average. Good stepping stone to better firms,After work drinks. Nice building. Desks that go up and down (exciting stuff).,Senior and mid-level management.\nCorporate identity.\nZero cohesiveness between the different departments / companies.\nC-Level board haven't got a clue on what's really going on or they just don't care.,Average. Good stepping stone to better firms. After work drinks. Nice building. Desks that go up and down (exciting stuff).. Senior and mid-level management.\nCorporate identity.\nZero cohesiveness between the different departments / companies.\nC-Level board haven't got a clue on what's really going on or they just don't care.,0.009715,0.232590,0.757695,Positive Sentiment
8,Octopus Investments,5,Amazing place to work!,"Octopus has such a great energy about it as a business, its people are dynamic, driven but also friendly and transparent. The energy within the business means that you are cons

In [16]:
# Overall Sentiment Analysis of Octopus Investments
octopus_investments_reviews_comp = octopus_investments_reviews[['firm','Negative Sentiment', 'Neutral Sentiment', 'Positive Sentiment']].copy()
agg_octopus_investments_reviews = octopus_investments_reviews_comp.groupby('firm', as_index=False).mean()

sentiment_df2 = agg_octopus_investments_reviews[["Negative Sentiment", "Neutral Sentiment", "Positive Sentiment"]]
agg_octopus_investments_reviews['Overall Sentiment'] = sentiment_df2.idxmax(axis=1)

In [17]:
display(HTML(agg_octopus_investments_reviews.to_html()))

,firm,Negative Sentiment,Neutral Sentiment,Positive Sentiment,Overall Sentiment
0,Octopus Investments,0.115829,0.166659,0.717512,Positive Sentiment


In [18]:
# Sentiment Analysis for each review of Octopus Investments - Save to csv
octopus_investments_reviews.to_csv (r'/Users/hpatel31/Library/CloudStorage/OneDrive-KPMG/Documents/oct_invest_sent_analysis.csv', index = None, header=True)

# Overall Sentiment Analysis of Octopus Investments - Save to csv
agg_octopus_investments_reviews.to_csv (r'/Users/hpatel31/Library/CloudStorage/OneDrive-KPMG/Documents/agg_oct_invest_sent_analysis.csv', index = None, header=True)

## Octopus Investment Google News - Sentiment Analysis

In [19]:
fin = "ProsusAI/finbert"

fin_tokenizer = AutoTokenizer.from_pretrained(fin)
fin_model = AutoModelForSequenceClassification.from_pretrained(fin)

def get_sentiment_financial(
    text
):
    model_input = fin_tokenizer(
        text, padding=True, truncation=True, return_tensors="pt"
    )
    model_output = fin_model(**model_input)
    predictions = torch.nn.functional.softmax(model_output.logits, dim=-1)
    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()
    pred_list = [negative, neutral, positive]

    return pred_list

In [25]:
neg = []
neu = []
pos = []
for text in google_octopus_investments["Text"]:
    output = get_sentiment_financial(text = text)
    neg.append(output[1][0])
    neu.append(output[2][0])
    pos.append(output[0][0])

google_octopus_investments["Negative Sentiment"] = neg
google_octopus_investments["Neutral Sentiment"] = neu
google_octopus_investments["Positive Sentiment"] = pos

sentiment_df3 = google_octopus_investments[["Negative Sentiment", "Neutral Sentiment", "Positive Sentiment"]]
google_octopus_investments['Overall Sentiment'] = sentiment_df3.idxmax(axis=1)
display(HTML(google_octopus_investments.to_html()))

,Title,Description,Text,Negative Sentiment,Neutral Sentiment,Positive Sentiment,Overall Sentiment
0,Octopus Australia buys Dulacca Wind Farm,"Octopus Investments has made its fourth investment in Australia, acquiring the Dulacca Wind Farm from global renewable energy company RES.","Octopus Australia buys Dulacca Wind Farm. Octopus Investments has made its fourth investment in Australia, acquiring the Dulacca Wind Farm from global renewable energy company RES.",0.497004,0.492804,0.010192,Negative Sentiment
1,Brisbane couple builds tech giant on the quiet,The hunt for the next Atlassian could be over after Paul and Sonia Stovell’s little-known Octopus Deploy banked one of the biggest ever local tech VC investments.,Brisbane couple builds tech giant on the quiet. The hunt for the next Atlassian could be over after Paul and Sonia Stovell’s little-known Octopus Deploy banked one of the biggest ever local tech VC investments.,0.846371,0.128849,0.024780,Negative Sentiment


In [27]:
google_octopus_investments_comp = google_octopus_investments[['Title','Negative Sentiment', 'Neutral Sentiment', 'Positive Sentiment']].copy()
agg_google_octopus_investments = google_octopus_investments_comp.groupby('Title', as_index=False).mean()

sentiment_df4 = agg_google_octopus_investments[["Negative Sentiment", "Neutral Sentiment", "Positive Sentiment"]]
agg_google_octopus_investments['Overall Sentiment'] = sentiment_df4.idxmax(axis=1)

display(HTML(agg_google_octopus_investments.to_html()))

,Title,Negative Sentiment,Neutral Sentiment,Positive Sentiment,Overall Sentiment
0,Brisbane couple builds tech giant on the quiet,0.846371,0.128849,0.024780,Negative Sentiment
1,Octopus Australia buys Dulacca Wind Farm,0.497004,0.492804,0.010192,Negative Sentiment


In [28]:
# Sentiment Analysis for each news article of Octopus Investments - Save to csv
google_octopus_investments.to_csv (r'/Users/hpatel31/Library/CloudStorage/OneDrive-KPMG/Documents/google_oct_invest_sent_analysis.csv', index = None, header=True)

# Overall Sentiment Analysis of Octopus Investments news articles - Save to csv
agg_google_octopus_investments.to_csv (r'/Users/hpatel31/Library/CloudStorage/OneDrive-KPMG/Documents/agg_google_oct_invest_sent_analysis.csv', index = None, header=True)
